### Blinker signals to couple QCoDeS StandardParameters

Here is a small example on how it could be done.

In [1]:
from blinker import signal
from qcodes.tests.instrument_mocks import DummyInstrument
from qcodes.instrument.parameter import StandardParameter
from functools import partial

Rather than actually extending the StandardParameter class, we'll just subclass it for now. 

In [2]:
class SignallingParameter(StandardParameter):
    """
    Development extension of StandardParameter to emit a signal
    on being set and to accept subscriptions ot other parameters
    """

    def __init__(self, name, label=None, unit=None,
                 set_cmd=None, get_cmd=None,
                 dependencies=None, **kwargs):
        """
        dependencies is a dict
        {signal: function}
        """
        super().__init__(name, label=label, unit=unit,
                         set_cmd=set_cmd, get_cmd=get_cmd, **kwargs)

        self.wasset = signal('{}-was-set'.format(self.name))
        self.set = self.make_set_emit()

        if dependencies is not None:

            self._depfuns = [None]

            for ii, (sign, func) in enumerate(dependencies.items()):
                self._depfuns[ii] = partial(func, self)
                sign.connect(self._depfuns[ii])

    def make_set_emit(self):
        def wrapper(arg):
            self._validate_and_set(arg)
            self.wasset.send(self, val=arg)  # First arg must have .name
        return wrapper

The idea is then that we, when adding parameters to the instrument, provide a dict of signals and functions.

#### Example 1: A voltage and a voltage range

We have a DAC with a voltage and a voltage range. When the voltage range is set, the physical voltage might be truncated. We'll make the parameters reflect this.

In [3]:
dac = DummyInstrument(name='dac')

dac.add_parameter('voltage_range',
                  label='Vrange',
                  unit='V',
                  set_cmd=lambda x: x,
                  parameter_class=SignallingParameter)

# The user must supply the logic (how could we possibly guess it?)
# The call signature should always be this
def check_with_range(inst, sender, **kw):
    """
    Function to check if voltage is out of range and if so truncate
    accordingly
    """
    value = kw['val']
    if inst.get_latest() > value:
        inst.set(value)  # perhaps inst._save_val would suffice
        
dac.add_parameter('volt',
                  label='Voltage',
                  unit='V',
                  set_cmd=lambda x: x,  # This should of course be a custom function checking voltage_range
                  dependencies={dac.voltage_range.wasset:  # <---- HERE!
                                check_with_range},
                  parameter_class=SignallingParameter)

In [4]:
# Now things work as expected
dac.volt(10)
print('Voltage is {}'.format(dac.volt.get_latest()))
dac.voltage_range(5)
print('Voltage is {}'.format(dac.volt.get_latest()))
dac.voltage_range(7)
print('Voltage is {}'.format(dac.volt.get_latest()))

Voltage is 10
Voltage is 5
Voltage is 5
